In [171]:
# Traffic prediction ML project
# Imports
import pandas as pd
import numpy as np


In [172]:
'''
Feature engineering notes:
    - Time and date
    - Combine Vehicle continuing direction and vehicle going direction. Oncoming/incoming or same direction
    - The weather and surface cond columns may help each other...
    - Modify Light so that we replace 'Dark -- Unknown Lighting' with 'Dark'
    
'''

"\nFeature engineering notes:\n    - Time and date\n    - Combine Vehicle continuing direction and vehicle going direction. Oncoming/incoming or same direction\n    - The weather and surface cond columns may help each other...\n    - Modify Light so that we replace 'Dark -- Unknown Lighting' with 'Dark'\n    \n"

In [173]:
data = pd.read_csv("traffic.csv")
# data.shape - 172,105 rows, 43 columns
# data['Off-Road Description'].unique() # Determine that the values in this column is not important
# data['Driverless Vehicle'].unique() # No driverless vehicles, so column is unneeded
# data['Parked Vehicle'].value_counts() # 2,600 parked vehicles got an in accident. Might be interesting to see if there's a correlation between that and the severity of the accident.
# data['Related Non-Motorist'].value_counts() - There's only about 5000 fields here, it may be too little to make a discernible impact on the dataset.




# Look into what the cross street and route type means
# DROP Report Number, Local Case Number, Off-road description, Road name, Cross-street name, Driverless Vehicle, Municipality (many missing values),  Related Non-Motorist, Non-Motorist Substance Abuse, Person ID, Circumstance (??), Vehicle ID, Vehicle Make, Vehicle Model, Equiptment Problems, Location (as it combines Lat & Long)

/var/folders/xp/ph5sj5y90_j0747lx52mp81r0000gn/T/ipykernel_88451/60067504.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("traffic.csv")


In [174]:
data['Collision Type'].value_counts()
# 12% other, 717 unknown. We may want to use similar-case imputation

data['Non-Motorist Substance Abuse'].value_counts()
# There are only ~200 data points that actually had substance abuse, so this should be dropped.

data['Driver Distracted By'].value_counts()
# 1/5th of the dataset is missing, over half has drivers that were not distracted. Maybe there's a correlation between distracted driving and severity? Maybe we can train a subset of the data on this to reveal more interesting patterns. Or, we could inject and say that half of the missing is distracted and the other half is not distracted. Or, maybe we should just drop it.

data['Drivers License State'].value_counts()
# correlation between driving out of state lines and being at fault? are drivers closer to home truly more likely to get in an accident? Outliers - FM, NS, MP, SK, etc.

data['Vehicle Movement'].value_counts()
# We need to get creative with our encoding, such as moving quickly, moving at a slower rate, or manuvering (parking, passing, turning, etc)

# This is our class label. We need to OHC different categories.
data['Injury Severity'].value_counts()
# the majority of the dataset isn't injured, fortunately.
# we will just keep the 5 predictors
    # No injury
    # Possible
    # Minor
    # Serious
    # Fatal :(

Injury Severity
NO APPARENT INJURY          141185
POSSIBLE INJURY              17482
SUSPECTED MINOR INJURY       11870
SUSPECTED SERIOUS INJURY      1415
FATAL INJURY                   153
Name: count, dtype: int64

In [175]:
data['Vehicle Dir Changed'] = np.where(data['Vehicle Continuing Dir'] == data['Vehicle Going Dir'], 'No', 'Yes')
data['Vehicle Multiple Impacts'] = np.where(data['Vehicle First Impact Location'] == data['Vehicle Second Impact Location'], 'No', 'Yes')
data = data.drop(columns = ['Agency Name', 'Report Number', 'Local Case Number', 'Off-Road Description', 'Road Name', 'Cross-Street Name', 'Driverless Vehicle', 'Municipality', 'Non-Motorist Substance Abuse', 'Person ID', 'Circumstance', 'Vehicle ID', 'Vehicle Make', 'Vehicle Model', 'Equipment Problems', 'Location','Driverless Vehicle', 'Vehicle Continuing Dir', 'Vehicle Going Dir', 'Route Type', 'Vehicle First Impact Location', 'Vehicle Second Impact Location'], axis = 1)
data.head()

,ACRS Report Type,Crash Date/Time,Cross-Street Type,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Vehicle Damage Extent,Vehicle Body Type,Vehicle Movement,Speed Limit,Parked Vehicle,Vehicle Year,Latitude,Longitude,Vehicle Dir Changed,Vehicle Multiple Impacts
0,Property Damage Crash,05/31/2019 03:00:00 PM,NaN,NaN,OTHER,CLEAR,NaN,DAYLIGHT,NaN,UNKNOWN,...,SUPERFICIAL,PASSENGER CAR,PARKING,15,No,2004,39.150044,-77.063089,No,No
1,Property Damage Crash,07/21/2023 05:59:00 PM,Unknown,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,DISABLING,PASSENGER CAR,MAKING LEFT TURN,40,No,2011,39.159264,-77.219025,Yes,Yes
2,Property Damage Crash,07/20/2023 03:10:00 PM,Maryland (State),NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,FUNCTIONAL,PICKUP TRUCK,ACCELERATING,35,No,2019,39.109535,-77.075806,No,No
3,Property Damage Crash,07/23/2023 12:10:00 PM,County,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,FUNCTIONAL,PASSENGER CAR,STARTING FROM LANE,40,No,2016,39.190149,-77.266766,No,No
4,Property Damage Crash,07/24/2023 06:10:00 AM,County,NaN,HEAD ON LEFT TURN,CLOUDY,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,FUNCTIONAL,(SPORT) UTILITY VEHICLE,MAKING LEFT TURN,35,No,2016,39.172558,-77.203745,Yes,No


In [176]:
# Dealing with Date/Time column
# Monday is 0, so SUNDAY is 6
import datetime
new_weekdays = []
for i in data['Crash Date/Time'] : 
    list = i.split(' ')[0].split('/')
    obj = datetime.datetime(int(list[2]), int(list[0]), int(list[1]))
    new_weekdays.append(obj.weekday())

data['Crash Date/Time'] = new_weekdays
df_encoded = pd.get_dummies(data['Crash Date/Time'], columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
# for num in range(len(days)) :
#     df_encoded[days[num]] = new_weekdays[num]
data = pd.concat([data, df_encoded] , axis=1)
data.head()


,ACRS Report Type,Crash Date/Time,Cross-Street Type,Related Non-Motorist,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,...,Longitude,Vehicle Dir Changed,Vehicle Multiple Impacts,0,1,2,3,4,5,6
0,Property Damage Crash,4.0,NaN,NaN,OTHER,CLEAR,NaN,DAYLIGHT,NaN,UNKNOWN,...,-77.063089,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Property Damage Crash,4.0,Unknown,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,-77.219025,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Property Damage Crash,3.0,Maryland (State),NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,-77.075806,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Property Damage Crash,6.0,County,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,...,-77.266766,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Property Damage Crash,0.0,County,NaN,HEAD ON LEFT TURN,CLOUDY,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,...,-77.203745,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
